In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

# Set up Spotify API credentials
client_id = "0054a24f2fc643c69d56d020dd5f70be"
client_secret = "98b4a4b772ad4eca934a92ca60c246a0"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Define a function to get an artist's top tracks
def get_artist_top_tracks(artist_name, num_tracks=50):
    artist_results = sp.search(q=f"artist:{artist_name}", type="artist")
    if artist_results['artists']['items']:
        artist_id = artist_results['artists']['items'][0]['id']
        top_tracks = sp.artist_top_tracks(artist_id)['tracks'][:num_tracks]
        
        return top_tracks
    else:
        print(f"No artist found for '{artist_name}'")
        return []

# Define a function to get audio features for a list of tracks
# def get_tracks_audio_features(tracks):
#     track_ids = [track['id'] for track in tracks]
#     audio_features = sp.audio_features(track_ids)
#     return audio_features
# Define a function to get audio features for a list of tracks
def get_tracks_audio_features(tracks):
    track_ids = [track['id'] for track in tracks]
    audio_features = sp.audio_features(track_ids)
    # Fetch popularity for each track
    track_info = sp.tracks(track_ids)['tracks']
    # Create a dictionary to map track IDs to popularity scores
    popularity_dict = {track['id']: track['popularity'] for track in track_info}
    # Add popularity to the audio features
    for af in audio_features:
        af['popularity'] = popularity_dict[af['id']]
    return audio_features

with open("artist.txt", "r") as file:
    artist_names = [line.strip() for line in file if line.strip()]
artist_names=list(set(artist_names))
# Create empty lists to store track data
track_names = []
artist_names_list = []
album_names = []
track_ids = []
audio_features_list = []

# Loop through each artist and get their top tracks and audio features
for artist_name in artist_names:
    top_tracks = get_artist_top_tracks(artist_name)
    if top_tracks:
        track_names.extend([track['name'] for track in top_tracks])
        artist_names_list.extend([artist_name] * len(top_tracks))
        album_names.extend([track['album']['name'] for track in top_tracks])
        track_ids.extend([track['id'] for track in top_tracks])
        audio_features_list.extend(get_tracks_audio_features(top_tracks))

# Create a pandas DataFrame from the collected data
data = {
    'track_name': track_names,
    'artist_name': artist_names_list,
    'album_name': album_names,
    'track_id': track_ids
}

for feature in audio_features_list[0].keys():
    data[feature] = [af[feature] for af in audio_features_list]

df = pd.DataFrame(data)
def get_release_year(track_id):
    try:
        # Fetch track information
        track_data = client.track(track_id)
        # Extract release year from album information
        release_date = track_data["album"]["release_date"]
        # Split date string and get year
        release_year = release_date.split("-")[0]
        return release_year
    except spotipy.exceptions.SpotifyException as e:
        print(f"Error getting release year for track ID {track_id}: {e}")
        return None  # Handle cases where track data is unavailable
# Create a new column for release year
df["release_year"] = df["track_id"].apply(get_release_year)

# Handle potential errors and missing values
df["release_year"].fillna(-1, inplace=True)  # Fill missing values with -1 (or any other indicator)
df.release_year = df.release_year.astype(int)

# Export the DataFrame to a CSV file
df.to_csv('top_tracks_data_new.csv', index=False)